In [97]:
from yargy import rule, Parser
from yargy.predicates import gram, dictionary, type, eq
from yargy.interpretation import fact, attribute
from yargy import Parser, rule, or_
from yargy.pipelines import morph_pipeline
from dataclasses import dataclass
from typing import Optional
from yargy.relations import gnc_relation


In [98]:
# Определение структур фактов
@dataclass
class Entry:
    name: str
    birth_date: Optional[str]
    birth_place: Optional[str]


In [99]:
Entry = fact(
    'Entry',
    ['Name', 'BirthDate', 'BirthPlace']
)

In [100]:
Name = fact(
    'Name',
    ['first', 'last', 'middle']
)

BirthDate = fact(
    'BirthDate',
    ['date']
)

BirthPlace = fact(
    'BirthPlace',
    ['place']
)


In [101]:
# Правила для имени, даты рождения и места рождения
NAME = rule(
    gram("Name").interpretation(
        Name.first.inflected()
    ),
    gram("Surn").interpretation(
        Name.last.inflected()
    ),
    gram('Patr').optional().interpretation(
        Name.middle.normalized()
    )
).interpretation(
    Name
)

BORN_VERB = morph_pipeline([
    "родился",
    "родилась",
    "родился в", 
    "родилась в", 
    "появился на свет в",
    "появилась на свет в",
    "родился в городе"
])

BORN_YEAR_WORD = morph_pipeline([
    "родился",
    "родилась",
    "родился в", 
    "родилась в", 
    "появился на свет в",
    "появилась на свет в",
    "родился в городе",
    "в",
    "году"
])


In [102]:
from yargy import rule, and_, or_, Parser
from yargy.predicates import gte, lte, dictionary, normalized
from yargy.interpretation import fact

# Создаем факт BirthDate с атрибутами day, month и year
BirthDate = fact(
    'BirthDate',
    ['day', 'month', 'year']
)

MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'май',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}

DAY = and_(
    gte(1),
    lte(31)
)

MONTH = dictionary(MONTHS)

YEAR = and_(
    gte(1),
    lte(2023)
)

DATE_TEXT = rule(
    DAY.interpretation(
        BirthDate.day
    ).optional(),
    MONTH.interpretation(
        BirthDate.month
    ).optional(),
    YEAR.interpretation(
        BirthDate.year
    )
).interpretation(
    BirthDate
)

DATE_DDMMYYYY = rule(
    DAY.interpretation(
        BirthDate.day
    ).optional(),
    '.',
    DAY.interpretation(
        BirthDate.month
    ).optional(),
    '.',
    YEAR.interpretation(
        BirthDate.year
    )
).interpretation(
    BirthDate
)

In [103]:
BIRTH_DATE = or_(
    DATE_DDMMYYYY,
    DATE_TEXT
).interpretation(
    BirthDate
)

BIRTH_PLACE = rule(
    gram('Geox').interpretation(
        BirthPlace.place.normalized()
    )
).interpretation(
    BirthPlace
)


In [104]:
# # Связь между элементами правил
# gnc = gnc_relation()
# ExtendedName = Name.match(gnc)
# BirthDate = BirthDate.match(gnc)
# BirthPlace = BirthPlace.match(gnc)

In [105]:

# Комбинированное правило
S = rule(
    NAME.interpretation(
        Entry.Name
    ),
    BORN_VERB,
    BIRTH_PLACE.optional().interpretation(
        Entry.BirthPlace
    ),
    BORN_YEAR_WORD.optional(),
    BIRTH_DATE.optional().interpretation(
        Entry.BirthDate
    )
).interpretation(
    Entry
)


In [106]:
# Создание и применение парсера
parser = Parser(S)

In [107]:
try:
    with open('/Users/kseniiatokareva/Documents/nlp-2023/news.txt', 'r', encoding='utf-8') as file:
        for line in file:
            for match in parser.findall(line):
                if(match.fact.BirthDate or match.fact.BirthPlace):
                    print(match.fact)
                name_fact = match.fact.Name
                birth_date_fact = match.fact.BirthDate
                birth_place_fact = match.fact.BirthPlace

                entry = Entry(
                    Name=name_fact,
                    BirthDate=birth_date_fact if birth_date_fact else None,
                    BirthPlace=birth_place_fact if birth_place_fact else None
                )

                # print(entry)
                
                with open('/Users/kseniiatokareva/Documents/nlp-2023/output2.txt', 'a', encoding='utf-8') as f:
                    f.write(str(entry) + '\n')

except Exception as e:
    print(f"Ошибка: {e}")

Entry(Name=Name(first='андрей', last='курносенко', middle=None), BirthDate=None, BirthPlace=BirthPlace(place='севастополь'))
Entry(Name=Name(first='яковлевюрий', last='яковлев', middle=None), BirthDate=BirthDate(day=None, month=None, year='1928'), BirthPlace=BirthPlace(place='москва'))
Entry(Name=Name(first='николай', last='караченцов', middle=None), BirthDate=BirthDate(day='27', month='октября', year='1944'), BirthPlace=None)
Entry(Name=Name(first='сергей', last='довлатов', middle=None), BirthDate=BirthDate(day=None, month=None, year='1941'), BirthPlace=None)
Entry(Name=Name(first='аня', last='титов', middle=None), BirthDate=BirthDate(day=None, month=None, year='1984'), BirthPlace=None)
Entry(Name=Name(first='игорь', last='доценко', middle=None), BirthDate=BirthDate(day=None, month=None, year='1953'), BirthPlace=None)
Entry(Name=Name(first='андрей', last='сердюков', middle=None), BirthDate=BirthDate(day=None, month=None, year='1962'), BirthPlace=None)
Entry(Name=Name(first='анатолий',